In [1]:
from datetime import *

class Dataform:
  
  # Lista com os deparas de tipagem
  depara = {"NullType()": NullType(), "StringType()": StringType(), "BinaryType()": BinaryType(), "BooleanType()": BooleanType(), "DateType()": DateType(), "TimestampType()": TimestampType(),
"DecimalType()": DecimalType(), "DoubleType()": DoubleType(), "FloatType()": FloatType(), "ByteType()": ByteType(), "IntegerType()": IntegerType(), "LongType()": LongType(), "ShortType()": ShortType(),
"ArrayType(NullType())": ArrayType(NullType()), "ArrayType(StringType())": ArrayType(StringType()), "ArrayType(BinaryType())": ArrayType(BinaryType()), "ArrayType(BooleanType())": ArrayType(BooleanType()), "ArrayType(DateType())": ArrayType(DateType()), "ArrayType(TimestampType())": ArrayType(TimestampType()), "ArrayType(DecimalType())": ArrayType(DecimalType()), "ArrayType(DoubleType())": ArrayType(DoubleType()), "ArrayType(FloatType())": ArrayType(FloatType()), "ArrayType(ByteType())": ArrayType(ByteType()), "ArrayType(IntegerType())": ArrayType(IntegerType()), "ArrayType(LongType())": ArrayType(LongType()), 
"ArrayType(ShortType())": ArrayType(ShortType())}
  
  # Referência de processamento
  ref = str(datetime.today())[0:10]
  
  def __init__(self, Dataframe, Metadata=None):
    
    if not Metadata:
      self.Dataframe = Dataframe
      self.Metadata = []
      
      # Vamos inicializar a construção do metadata de cada variável
      for coluna in self.Dataframe.columns:
        info = {
          "nome": coluna,
          "origem": coluna,
          "tipo": "StringType()",
          "valor_se_nulo": None,
          "dominio": None,
          "transformacao": None,
          "versao": self.ref
        }
        self.Metadata.append(info)
    else:
        self.Metadata = Metadata
        
        # A partir de um dado metadados fornecido pelo usuário, iniciaremos o Dataforming
        # Etapa 01 - Pré seleção das colunas de origem
        infos_com_origem = [info for info in self.Metadata if info["origem"]]
        colunas = [info["origem"] for info in infos_com_origem]
        self.Dataframe = Dataframe.select(colunas)
        
        # Etapa 02 - Inicializa os módulos (Caster, Coalescer, Transformer, Domain, Namer)
        # Faremos primeiro os casos sem origem
        infos_sem_origem = [info for info in self.Metadata if not info["origem"]]
        infos_de_grau_maiorigual_um = []
        
        for info in infos_sem_origem:
          try:
            self.transformedAs({info["nome"]: info["transformacao"]}, True)
            self.coalescedAs({info["nome"]: info["valor_se_nulo"]}, True)
            self.castedAs({info["nome"]: self.depara[info["tipo"]]}, True)
            self.domainAs({info["nome"]: info["dominio"]}, True)
          except:
            infos_de_grau_maiorigual_um.append(info)
            
        # Agora os casos que possuem origem
        for info in infos_com_origem:
          self.transformedAs({info["origem"]: info["transformacao"]}, True)
          self.coalescedAs({info["origem"]: info["valor_se_nulo"]}, True)
          self.castedAs({info["origem"]: self.depara[info["tipo"]]}, True)
          self.domainAs({info["origem"]: info["dominio"]}, True)
          self.namedAs({info["origem"]: info["nome"]}, True)
          
        # Por fim executa os módulos para as variáveis de grau maior ou iguais a 1
        index = 0
        
        while infos_de_grau_maiorigual_um:
          info = infos_de_grau_maiorigual_um[index]
          
          try:
            self.transformedAs({info["nome"]: info["transformacao"]}, True)
            self.coalescedAs({info["nome"]: info["valor_se_nulo"]}, True)
            self.castedAs({info["nome"]: self.depara[info["tipo"]]}, True)
            self.domainAs({info["nome"]: info["dominio"]}, True)
            infos_de_grau_maiorigual_um.remove(info)
            index = 0
          except:
            index += 1

            
  # Método de atualização do metadados
  def __updateMetadata__(self, col_list, dict_list):
    # Verifica e insere informações novas
    for coluna in [nova_coluna for nova_coluna in col_list if nova_coluna not in self.Dataframe.columns]:
      self.Metadata.append({"nome": coluna, "origem": None, "tipo": "StringType()", "valor_se_nulo": None, "dominio": None, "transformacao": None, "versao": self.ref})
    
    # Atualiza de fato o metadados
    for info in self.Metadata:
      if info["nome"] in col_list:
        index = col_list.index(info["nome"])
        dict = dict_list[index]
        
        for chave in dict.keys():
          info[chave] = dict[chave]
      
      
  # Método de construção do dataframe para o metadados
  def __buildMetadataDF__(self):
    # Construção do header
    meta_header = []
    for chave in self.Metadata[0].keys():
      meta_header.append(chave)
    
    # Construção das linhas
    lst_rows = []
    for info in self.Metadata:
      row = []
      
      for valor in info.values():
        row.append(str(valor))
      lst_rows.append(row)
      
    # Constrói o dataframe
    meta_DF = spark.createDataFrame(lst_rows, meta_header)

    return meta_DF
    
    
  @property # Retorna o metadados
  def metadata(self):
    return self.Metadata
  
  
  @property # Retorna o dataframe
  def dataframe(self):
    return self.Dataframe
  
  
  # Módulo 01 - Alteração de tipagem dos campos
  def castedAs(self, dict, imported_metadata=False):
    lst_casted = []
    
    for coluna in self.Dataframe.columns:
      if coluna in dict.keys() and dict[coluna]:
        casted = col(coluna).cast(dict[coluna])
        aliased = casted.alias(coluna)
        if not imported_metadata:
          self.__updateMetadata__([coluna], [{"tipo": str(dict[coluna]) + "()", "versao": self.ref}])
      else:
        aliased = col(coluna)
      lst_casted.append(aliased)
      
    self.Dataframe = self.Dataframe.select(lst_casted)
    return self.Dataframe
  
  
  # Módulo 02 - Tratamento de valores nulos
  def coalescedAs(self, dict, imported_metadata=False):
    lst_coalesced = []
    
    for coluna in self.Dataframe.columns:
      if coluna in dict.keys():
        # Devemos checar primeiro se temos um valor a inserir ou uma expressão envolvendo uma ou mais colunas
        try:
          self.Dataframe.select(expr(dict[coluna]))
          coalesced = coalesce(col(coluna), expr(dict[coluna]))
        except:
          coalesced = coalesce(col(coluna), lit(dict[coluna]))
        finally:
          aliased = coalesced.alias(coluna)
          if not imported_metadata:
            self.__updateMetadata__([coluna], [{"valor_se_nulo": dict[coluna], "versao": self.ref}])
      else:
          aliased = col(coluna)
      lst_coalesced.append(aliased)
        
    self.Dataframe = self.Dataframe.select(lst_coalesced)
    return self.Dataframe
  
  
  # Módulo 03 - Captura ou inserção da listagem de domínios
  def domainAs(self, dict, imported_metadata=False):
    if not imported_metadata:
      for info in self.Metadata:
        if info["nome"] in dict.keys():
          # Verifica se necessitamos gerar este domínio através de um agrupamento na variável
          try:
            dominio = []
            valores = self.Dataframe.select(dict[info["nome"]]).distinct().collect()

            for valor in valores:
              dominio.append(valor[info["nome"]])
            info["dominio"] = dominio
          except:
            info["dominio"] = dict[info["nome"]]
          info["versao"] = self.ref
          
    return self.Metadata
  
  
  # Módulo 04 - Aplica transformações em colunas existentes ou cria colunas novas
  def transformedAs(self, dict, imported_metadata=False):
    lst_transformed = []
    
    # Caso 01 - Coluna já existente no dataframe
    for coluna in self.Dataframe.columns:
      if coluna in dict.keys() and dict[coluna]:
        transformed = expr(dict[coluna])
        aliased = transformed.alias(coluna)
        if not imported_metadata:
          self.__updateMetadata__([coluna], [{"transformacao": dict[coluna], "versao": self.ref}])
      else:
        aliased = col(coluna)
      lst_transformed.append(aliased)
        
    # Caso 02 - Uma nova coluna a ser criada
    for coluna in [nova_coluna for nova_coluna in dict.keys() if nova_coluna not in self.Dataframe.columns]:
      transformed = expr(dict[coluna]).cast(StringType())
      aliased = transformed.alias(coluna)
      lst_transformed.append(aliased)
      if not imported_metadata:
        self.__updateMetadata__([coluna], [{"transformacao": dict[coluna], "versao": self.ref}])
        
    self.Dataframe = self.Dataframe.select(lst_transformed)
    return self.Dataframe
  
  
  # Módulo 05 - Renomeação de variáveis
  def namedAs(self, dict, imported_metadata=False):
    lst_named = []
    
    for coluna in self.Dataframe.columns:
      if coluna in dict.keys() and dict[coluna]:
        aliased = col(coluna).alias(dict[coluna])
        if not imported_metadata:
          self.__updateMetadata__([coluna], [{"nome": dict[coluna], "versao": self.ref}])
      else:
        aliased = col(coluna)
      lst_named.append(aliased)
    
    self.Dataframe = self.Dataframe.select(lst_named)
    return self.Dataframe
  
  
  # Visualização do Metadados
  def viewMetadata(self, truncate=True):
    meta_DF = self.__buildMetadataDF__()
    meta_DF = meta_DF.select(col("nome").alias("Nome"), col("origem").alias("Origem"), col("tipo").alias("Tipo"), col("valor_se_nulo").alias("Valor/Expressão(SQL) se Nulo"), 
                             col("dominio").alias("Domínio"), col("transformacao").alias("Transformação(SQL)"), col("versao").alias("Versão"))
    
    return meta_DF.show(len(self.Metadata), truncate)
  
  
  # Exportação do metadados para um diretório especificado pelo usuário
  def saveMetadata(self, dir):
    meta_DF = self.__buildMetadataDF__()
    meta_DF.coalesce(1).write.mode("overwrite").format("json").save(dir)
    
    return "> Metadados exportado com SUCESSO para o diretório: {}.".format(dir)
  
  

In [2]:
def uploadMetadata(dir):
  import json
  
  # Lendo o metadados do diretório especificado
  metadados = []
  
  for linha in spark.sparkContext.textFile(dir).collect():
    info = json.loads(linha)

    # Faz o tratamento dos campos com conteúdo 'None'
    for chave in info:
      if info[chave] == "None":
        info[chave] = None
    metadados.append(info)
  
  return metadados

In [3]:
# Inicializando variáveis, funções e importando pacotes

from pyspark import StorageLevel
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import *

# Inicializa sessão spark
spark = SparkSession.builder.appName("dataform-desenv").getOrCreate()

In [4]:
# Leitura da base de entrada
origem_pouso = "/FileStore/tables/user_ct_test_collection_01_txt-4eeb3.gz"
destino_meta = "/FileStore/metadados/"

pouso = spark.read.option("delimiter", "\t").option("header", True).csv(origem_pouso).persist(StorageLevel.DISK_ONLY)
print("> Volumetria da base de entrada: {}".format(pouso.count()))

> Volumetria da base de entrada: 3558411

In [5]:
pouso.show(5)

+------+--------------------+-------------------+--------+--------+
AnonID| Query| QueryTime|ItemRank|ClickURL|
+------+--------------------+-------------------+--------+--------+
 142| rentdirect.com|2006-03-01 07:17:12| null| null|
 142|www.prescriptionf...|2006-03-12 12:31:06| null| null|
 142| staple.com|2006-03-17 21:19:29| null| null|
 142| staple.com|2006-03-17 21:19:45| null| null|
 142|www.newyorklawyer...|2006-03-18 08:02:58| null| null|
+------+--------------------+-------------------+--------+--------+
only showing top 5 rows

In [6]:
# Criando instância dataform
dataform = Dataform(pouso)

In [7]:
dataform.viewMetadata()

+---------+---------+------------+----------------------------+-------+------------------+----------+
 Nome| Origem| Tipo|Valor/Expressão(SQL) se Nulo|Domínio|Transformação(SQL)| Versão|
+---------+---------+------------+----------------------------+-------+------------------+----------+
 AnonID| AnonID|StringType()| None| None| None|2019-10-03|
 Query| Query|StringType()| None| None| None|2019-10-03|
QueryTime|QueryTime|StringType()| None| None| None|2019-10-03|
 ItemRank| ItemRank|StringType()| None| None| None|2019-10-03|
 ClickURL| ClickURL|StringType()| None| None| None|2019-10-03|
+---------+---------+------------+----------------------------+-------+------------------+----------+

In [8]:
tipos = {
  "AnonID": IntegerType(),
  "QueryTime": TimestampType(),
  "ItemRank": IntegerType()
}

dataform.castedAs(tipos)

Out[ 74 ]: DataFrame[AnonID: int, Query: string, QueryTime: timestamp, ItemRank: int, ClickURL: string]

In [9]:
dataform.viewMetadata()

+---------+---------+---------------+----------------------------+-------+------------------+----------+
 Nome| Origem| Tipo|Valor/Expressão(SQL) se Nulo|Domínio|Transformação(SQL)| Versão|
+---------+---------+---------------+----------------------------+-------+------------------+----------+
 AnonID| AnonID| IntegerType()| None| None| None|2019-10-03|
 Query| Query| StringType()| None| None| None|2019-10-03|
QueryTime|QueryTime|TimestampType()| None| None| None|2019-10-03|
 ItemRank| ItemRank| IntegerType()| None| None| None|2019-10-03|
 ClickURL| ClickURL| StringType()| None| None| None|2019-10-03|
+---------+---------+---------------+----------------------------+-------+------------------+----------+

In [10]:
se_nulos = {
  "ClickURL": "CASE WHEN ClickURL IS NULL THEN '' ELSE ClickURL END",
  "QueryTime": "0001-01-01 00:00:00"
}

dataform.coalescedAs(se_nulos)

Out[ 75 ]: DataFrame[AnonID: int, Query: string, QueryTime: string, ItemRank: int, ClickURL: string]

In [11]:
dataform.dataframe.filter(col("ClickURL") == "").show(5)

+------+--------------------+-------------------+--------+--------+
AnonID| Query| QueryTime|ItemRank|ClickURL|
+------+--------------------+-------------------+--------+--------+
 142| rentdirect.com|2006-03-01 07:17:12| null| |
 142|www.prescriptionf...|2006-03-12 12:31:06| null| |
 142| staple.com|2006-03-17 21:19:29| null| |
 142| staple.com|2006-03-17 21:19:45| null| |
 142|www.newyorklawyer...|2006-03-18 08:02:58| null| |
+------+--------------------+-------------------+--------+--------+
only showing top 5 rows

In [12]:
dataform.viewMetadata(False)

+---------+---------+---------------+----------------------------------------------------+-------+-------------+----------+
Nome |Origem |Tipo |Valor se Nulo |Domínio|Transformação|Versão |
+---------+---------+---------------+----------------------------------------------------+-------+-------------+----------+
AnonID |AnonID |IntegerType() |None |None |None |2019-09-29|
Query |Query |StringType() |None |None |None |2019-09-29|
QueryTime|QueryTime|TimestampType()|0001-01-01 00:00:00 |None |None |2019-09-29|
ItemRank |ItemRank |IntegerType() |None |None |None |2019-09-29|
ClickURL |ClickURL |StringType() |CASE WHEN ClickURL IS NULL THEN '' ELSE ClickURL END|None |None |2019-09-29|
+---------+---------+---------------+----------------------------------------------------+-------+-------------+----------+

In [13]:
dominios = {
  "AnonID": "Dominio infinito.",
  "ItemRank": "Dominio infinito."
}

dataform.domainAs(dominios)

Out[ 76 ]: [{'dominio': 'Dominio infinito.',
 'transformacao': None,
 'versao': '2019-10-03',
 'tipo': 'IntegerType()',
 'origem': 'AnonID',
 'valor_se_nulo': None,
 'nome': 'AnonID'},
 {'dominio': None,
 'transformacao': None,
 'versao': '2019-10-03',
 'tipo': 'StringType()',
 'origem': 'Query',
 'valor_se_nulo': None,
 'nome': 'Query'},
 {'dominio': None,
 'transformacao': None,
 'versao': '2019-10-03',
 'tipo': 'TimestampType()',
 'origem': 'QueryTime',
 'valor_se_nulo': '0001-01-01 00:00:00',
 'nome': 'QueryTime'},
 {'dominio': 'Dominio infinito.',
 'transformacao': None,
 'versao': '2019-10-03',
 'tipo': 'IntegerType()',
 'origem': 'ItemRank',
 'valor_se_nulo': None,
 'nome': 'ItemRank'},
 {'dominio': None,
 'transformacao': None,
 'versao': '2019-10-03',
 'tipo': 'StringType()',
 'origem': 'ClickURL',
 'valor_se_nulo': "CASE WHEN ClickURL IS NULL THEN '' ELSE ClickURL END",
 'nome': 'ClickURL'}]

In [14]:
trans = {
  "QueryDate": "CAST(QueryTime AS DATE)",
  "Query": "UPPER(Query)"
}

dataform.transformedAs(trans)
dataform.coalescedAs({"QueryDate": "0001-01-01"})
dataform.castedAs({"QueryDate": DateType()})

Out[ 77 ]: DataFrame[AnonID: int, Query: string, QueryTime: string, ItemRank: int, ClickURL: string, QueryDate: date]

In [15]:
dataform.dataframe.show(5)

+------+--------------------+-------------------+--------+--------+----------+
AnonID| Query| QueryTime|ItemRank|ClickURL| QueryDate|
+------+--------------------+-------------------+--------+--------+----------+
 142| RENTDIRECT.COM|2006-03-01 07:17:12| null| |2006-03-01|
 142|WWW.PRESCRIPTIONF...|2006-03-12 12:31:06| null| |2006-03-12|
 142| STAPLE.COM|2006-03-17 21:19:29| null| |2006-03-17|
 142| STAPLE.COM|2006-03-17 21:19:45| null| |2006-03-17|
 142|WWW.NEWYORKLAWYER...|2006-03-18 08:02:58| null| |2006-03-18|
+------+--------------------+-------------------+--------+--------+----------+
only showing top 5 rows

In [16]:
dataform.viewMetadata()

+---------+---------+---------------+--------------------+-----------------+--------------------+----------+
 Nome| Origem| Tipo| Valor se Nulo| Domínio| Transformação| Versão|
+---------+---------+---------------+--------------------+-----------------+--------------------+----------+
 AnonID| AnonID| IntegerType()| None|Dominio infinito.| None|2019-09-30|
 Query| Query| StringType()| None| None| UPPER(Query)|2019-09-30|
QueryTime|QueryTime|TimestampType()| 0001-01-01 00:00:00| None| None|2019-09-30|
 ItemRank| ItemRank| IntegerType()| None|Dominio infinito.| None|2019-09-30|
 ClickURL| ClickURL| StringType()|CASE WHEN ClickUR...| None| None|2019-09-30|
QueryDate| None| DateType()| 0001-01-01| None|CAST(QueryTime AS...|2019-09-30|
+---------+---------+---------------+--------------------+-----------------+--------------------+----------+

In [17]:
nomes = {
  "AnonID": "id",
  "Query": "query",
  "QueryTime": "query_time",
  "ItemRank": "item_rank",
  "ClickURL": "click_url",
  "QueryDate": "query_date"
}

dataform.namedAs(nomes)

Out[ 78 ]: DataFrame[id: int, query: string, query_time: string, item_rank: int, click_url: string, query_date: date]

In [18]:
dataform.viewMetadata(False)

+----------+---------+---------------+----------------------------------------------------+-----------------+-----------------------+----------+
Nome |Origem |Tipo |Valor/Expressão(SQL) se Nulo |Domínio |Transformação(SQL) |Versão |
+----------+---------+---------------+----------------------------------------------------+-----------------+-----------------------+----------+
id |AnonID |IntegerType() |None |Dominio infinito.|None |2019-10-03|
query |Query |StringType() |None |None |UPPER(Query) |2019-10-03|
query_time|QueryTime|TimestampType()|0001-01-01 00:00:00 |None |None |2019-10-03|
item_rank |ItemRank |IntegerType() |None |Dominio infinito.|None |2019-10-03|
click_url |ClickURL |StringType() |CASE WHEN ClickURL IS NULL THEN '' ELSE ClickURL END|None |None |2019-10-03|
query_date|None |DateType() |0001-01-01 |None |CAST(QueryTime AS DATE)|2019-10-03|
+----------+---------+---------------+----------------------------------------------------+-----------------+-----------------------+----------+

In [19]:
trans = {
  "query_day": "DAY(query_date)"
}

dataform.transformedAs(trans)

Out[ 79 ]: DataFrame[id: int, query: string, query_time: string, item_rank: int, click_url: string, query_date: date, query_day: string]

In [20]:
dataform.viewMetadata(False)

+----------+---------+---------------+----------------------------------------------------+-----------------+-----------------------+----------+
Nome |Origem |Tipo |Valor/Expressão(SQL) se Nulo |Domínio |Transformação(SQL) |Versão |
+----------+---------+---------------+----------------------------------------------------+-----------------+-----------------------+----------+
id |AnonID |IntegerType() |None |Dominio infinito.|None |2019-10-03|
query |Query |StringType() |None |None |UPPER(Query) |2019-10-03|
query_time|QueryTime|TimestampType()|0001-01-01 00:00:00 |None |None |2019-10-03|
item_rank |ItemRank |IntegerType() |None |Dominio infinito.|None |2019-10-03|
click_url |ClickURL |StringType() |CASE WHEN ClickURL IS NULL THEN '' ELSE ClickURL END|None |None |2019-10-03|
query_date|None |DateType() |0001-01-01 |None |CAST(QueryTime AS DATE)|2019-10-03|
query_day |None |StringType() |None |None |DAY(query_date) |2019-10-03|
+----------+---------+---------------+----------------------------------------------------+-----------------+-----------------------+----------+

In [21]:
dataform.saveMetadata(destino_meta)

Out[ 81 ]: '> Metadados exportado com SUCESSO para o diretório: /FileStore/metadados/.'

In [22]:
spark.read.json(destino_meta).show()

+-----------------+----------+---------+---------------+--------------------+--------------------+----------+
 dominio| nome| origem| tipo| transformacao| valor_se_nulo| versao|
+-----------------+----------+---------+---------------+--------------------+--------------------+----------+
Dominio infinito.| id| AnonID| IntegerType()| None| None|2019-10-03|
 None| query| Query| StringType()| UPPER(Query)| None|2019-10-03|
 None|query_time|QueryTime|TimestampType()| None| 0001-01-01 00:00:00|2019-10-03|
Dominio infinito.| item_rank| ItemRank| IntegerType()| None| None|2019-10-03|
 None| click_url| ClickURL| StringType()| None|CASE WHEN ClickUR...|2019-10-03|
 None|query_date| None| DateType()|CAST(QueryTime AS...| 0001-01-01|2019-10-03|
 None| query_day| None| StringType()| DAY(query_date)| None|2019-10-03|
+-----------------+----------+---------+---------------+--------------------+--------------------+----------+

In [23]:
metadados = uploadMetadata(destino_meta)
metadados

Out[ 83 ]: [{'dominio': 'Dominio infinito.',
 'transformacao': None,
 'versao': '2019-10-03',
 'tipo': 'IntegerType()',
 'origem': 'AnonID',
 'valor_se_nulo': None,
 'nome': 'id'},
 {'dominio': None,
 'transformacao': 'UPPER(Query)',
 'versao': '2019-10-03',
 'tipo': 'StringType()',
 'origem': 'Query',
 'valor_se_nulo': None,
 'nome': 'query'},
 {'dominio': None,
 'transformacao': None,
 'versao': '2019-10-03',
 'tipo': 'TimestampType()',
 'origem': 'QueryTime',
 'valor_se_nulo': '0001-01-01 00:00:00',
 'nome': 'query_time'},
 {'dominio': 'Dominio infinito.',
 'transformacao': None,
 'versao': '2019-10-03',
 'tipo': 'IntegerType()',
 'origem': 'ItemRank',
 'valor_se_nulo': None,
 'nome': 'item_rank'},
 {'dominio': None,
 'transformacao': None,
 'versao': '2019-10-03',
 'tipo': 'StringType()',
 'origem': 'ClickURL',
 'valor_se_nulo': "CASE WHEN ClickURL IS NULL THEN '' ELSE ClickURL END",
 'nome': 'click_url'},
 {'dominio': None,
 'transformacao': 'CAST(QueryTime AS DATE)',
 'versao': '2019-10-03',
 'tipo': 'DateType()',
 'origem': None,
 'valor_se_nulo': '0001-01-01',
 'nome': 'query_date'},
 {'dominio': None,
 'transformacao': 'DAY(query_date)',
 'versao': '2019-10-03',
 'tipo': 'StringType()',
 'origem': None,
 'valor_se_nulo': None,
 'nome': 'query_day'}]

In [24]:
teste = Dataform(pouso, metadados)

In [25]:
teste.viewMetadata(False)

+----------+---------+---------------+----------------------------------------------------+-----------------+-----------------------+----------+
Nome |Origem |Tipo |Valor/Expressão(SQL) se Nulo |Domínio |Transformação(SQL) |Versão |
+----------+---------+---------------+----------------------------------------------------+-----------------+-----------------------+----------+
id |AnonID |IntegerType() |None |Dominio infinito.|None |2019-10-03|
query |Query |StringType() |None |None |UPPER(Query) |2019-10-03|
query_time|QueryTime|TimestampType()|0001-01-01 00:00:00 |None |None |2019-10-03|
item_rank |ItemRank |IntegerType() |None |Dominio infinito.|None |2019-10-03|
click_url |ClickURL |StringType() |CASE WHEN ClickURL IS NULL THEN '' ELSE ClickURL END|None |None |2019-10-03|
query_date|None |DateType() |0001-01-01 |None |CAST(QueryTime AS DATE)|2019-10-03|
query_day |None |StringType() |None |None |DAY(query_date) |2019-10-03|
+----------+---------+---------------+----------------------------------------------------+-----------------+-----------------------+----------+

In [26]:
teste.dataframe.show(5)

+---+--------------------+-------------------+---------+---------+----------+---------+
 id| query| query_time|item_rank|click_url|query_date|query_day|
+---+--------------------+-------------------+---------+---------+----------+---------+
142| RENTDIRECT.COM|2006-03-01 07:17:12| null| |2006-03-01| 1|
142|WWW.PRESCRIPTIONF...|2006-03-12 12:31:06| null| |2006-03-12| 12|
142| STAPLE.COM|2006-03-17 21:19:29| null| |2006-03-17| 17|
142| STAPLE.COM|2006-03-17 21:19:45| null| |2006-03-17| 17|
142|WWW.NEWYORKLAWYER...|2006-03-18 08:02:58| null| |2006-03-18| 18|
+---+--------------------+-------------------+---------+---------+----------+---------+
only showing top 5 rows

In [27]:
teste.dataframe.printSchema()

root
-- id: integer (nullable = true)
-- query: string (nullable = true)
-- query_time: timestamp (nullable = true)
-- item_rank: integer (nullable = true)
-- click_url: string (nullable = true)
-- query_date: date (nullable = true)
-- query_day: string (nullable = true)

In [28]:
cast = {
  "item_rank": StringType()
}

teste.castedAs(cast)

Out[ 88 ]: DataFrame[id: int, query: string, query_time: timestamp, item_rank: string, click_url: string, query_date: date, query_day: string]

In [29]:
teste.viewMetadata()

+----------+---------+---------------+----------------------------+-----------------+--------------------+----------+
 Nome| Origem| Tipo|Valor/Expressão(SQL) se Nulo| Domínio| Transformação(SQL)| Versão|
+----------+---------+---------------+----------------------------+-----------------+--------------------+----------+
 id| AnonID| IntegerType()| None|Dominio infinito.| None|2019-10-03|
 query| Query| StringType()| None| None| UPPER(Query)|2019-10-03|
query_time|QueryTime|TimestampType()| 0001-01-01 00:00:00| None| None|2019-10-03|
 item_rank| ItemRank| StringType()| None|Dominio infinito.| None|2019-10-03|
 click_url| ClickURL| StringType()| CASE WHEN ClickUR...| None| None|2019-10-03|
query_date| None| DateType()| 0001-01-01| None|CAST(QueryTime AS...|2019-10-03|
 query_day| None| StringType()| None| None| DAY(query_date)|2019-10-03|
+----------+---------+---------------+----------------------------+-----------------+--------------------+----------+